In [1]:
import sys
sys.path.append('../../preprocess_assets/')

In [2]:
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding
import tensorflow as tf
from features_extraction import *
from data_shuffling_split import *
from ara_vec_preprocess_configs import *
from ml_modeling import *
from keras_models import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
strat_train_set = read_file("train/strat_train_set.csv")
strat_train_set.head()

,label,text
0,1,مع انتشار الامراض ومنها الكورونا اليكم النصيحه...
1,0,1 دراسه بحثيه صادره معهد الدراسات الاستراتيجي...
2,1,بدات في مدينه وهان في الصين انتقلت لبعض المقاط...
3,1,coronarvirus CoronaOutbreak فيروس_كورونا
4,1,الاداره_العامه_للطيران_المدني اجراءات وقاءيه ح...


In [4]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

The number of instances in the training data after StratifiedShuffleSplit are:  8643
The number of instances in the testing data after StratifiedShuffleSplit are:   177
The number of trainin instances:  8643
The number of validation instances:  177
The number of trainin labels :  8643
The number of validation labels :  177


In [5]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

fullgram_x_train_text_tokenized = get_all_ngrams(x_train_text_tokenized)
print("full gram tokenization : \n", fullgram_x_train_text_tokenized[:3])
print("="*50)
fullgram_x_val_text_tokenized = get_all_ngrams(x_val_text_tokenized)
print("full gram tokenization : \n", fullgram_x_val_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['الصين تشيد مستشفي لعلاج مرضي ', 'ومع ان تصنيف الفيروس ورقعه انتشاره تعدت المرحله السادسه فتره الا ان منظمه الصحه العالميه لم تعلن حتي تفشي وباء عالمي قد يكون خوفا انتشار حاله هلع غير مبرر حول العالم وما قد ينتج عواقب اقتصاديه واجتماعيه', 'اتفاقيه الصين 300 الف برميل نفط يوميا لصين ولمده عشر اعوام مقابل اعمار العراق خلال سنتين سيكون العراق اكبر مركز تجاري عالمي يربط الشرق اوروبا عبر شبكه خطوط سكك حديد مما يءدي افلاس موانء دبي و خساءر فادحه لمضيق السويس وكسر الهيمنه الامريكيه اتفاقيه_الصين_مشاريع_واعمار']
After Tokenization : 
 [['الصين', 'تشيد', 'مستشفي', 'لعلاج', 'مرضي'], ['ومع', 'ان', 'تصنيف', 'الفيروس', 'ورقعه', 'انتشاره', 'تعدت', 'المرحله', 'السادسه', 'فتره', 'الا', 'ان', 'منظمه', 'الصحه', 'العالميه', 'لم', 'تعلن', 'حتي', 'تفشي', 'وباء', 'عالمي', 'قد', 'يكون', 'خوفا', 'انتشار', 'حاله', 'هلع', 'غير', 'مبرر', 'حول', 'العالم', 'وما', 'قد', 'ينتج', 'عواقب', 'اقتصاديه', 'واجتماعيه'], ['اتفاقيه', 'الصين', '300', 'الف', 'برميل', 'نفط', 'يوميا', 'لصين', 'ولمده', 'ع

# Our CBOW Word2Vec Model

In [6]:
our_word2vec_model = load_word2vec_model("../word2vec_models/rezk/cbow/continuous_bow_fullgram_vec_size_300-d_min_count_100")

In [7]:
number_of_features = 300
max_len_str = 237
word2vec_path = "rezk/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(8643, 237, 300)
(8643, 71100)
[-0.4805   0.1469   0.515   -0.0917   0.1716  -0.5327  -0.00957 -0.2554
  0.1427  -0.1697   0.1628  -0.1864  -0.423    0.1849  -0.01895  0.2593
  0.0554  -0.202   -0.9077  -0.1611  -0.2866  -0.1271  -0.119    0.0774
  0.00858 -0.03262  0.01156 -0.05792  0.09357 -0.303   -0.1738  -0.2012
 -0.252   -0.10443  0.1705   0.2007  -0.3098   0.0258  -0.2323  -0.282
 -0.514   -0.3896   0.1611  -0.1481   0.0856  -0.01016  1.23     0.2974
  0.3635  -0.679  ]
(177, 237, 300)
(177, 71100)
[ 0.3225  -0.2566   0.346   -0.258    0.2615   0.2467  -0.4841   0.10724
  0.02872 -0.7285  -0.1974   0.3418  -0.516    0.1583   0.7427  -0.02202
 -0.1685   0.1998  -0.1421  -0.1327   0.3577  -0.0834  -0.4927  -0.3665
 -0.562    0.05362 -0.06793 -0.1676  -0.278   -0.333    0.384    0.8403
  0.0937   0.419    0.2471  -0.519   -0.1809   0.2449  -0.432    0.1971
  0.148   -0.2384   0.4375  -0.03372  0.2788   0.04242  0.683    0.7944
 -0.0239   0.3176 ]


# Our CBOW Word2Vec Model

# With  Rmsprob and  Batch Normalization

In [8]:
callbacks_ = keras_callbacks(word2vec_type="rezk_cbow_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

./../ml_models_saved/dl_models/tensor_logs/run_2022_05_09_23_38_49_rezk_cbow_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_


2022-05-09 23:38:49.299756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:38:49.309677: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:38:49.311504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:38:49.753498: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:38:49.755288: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 237, 300)         1200      
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 237, 25)           32600     
                                                                 
 batch_normalization_1 (Batc  (None, 237, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 237, 25)           0         
                                                                 
 flatten (Flatten)           (None, 5925)              0         
                                                                 
 dense (Dense)               (None, 18)                1

In [9]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10


2022-05-09 23:39:06.356560: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


271/271 [==============================] - 16s 40ms/step - loss: 1.0416 - accuracy: 0.5688 - val_loss: 2.3733 - val_accuracy: 0.3446 - lr: 5.0000e-05
Epoch 2/10
271/271 [==============================] - 10s 37ms/step - loss: 0.5393 - accuracy: 0.7493 - val_loss: 0.4666 - val_accuracy: 0.7571 - lr: 5.0000e-05
Epoch 3/10
271/271 [==============================] - 10s 37ms/step - loss: 0.4138 - accuracy: 0.8134 - val_loss: 0.3850 - val_accuracy: 0.8305 - lr: 5.0000e-05
Epoch 4/10
271/271 [==============================] - 10s 36ms/step - loss: 0.3545 - accuracy: 0.8460 - val_loss: 0.3605 - val_accuracy: 0.8588 - lr: 5.0000e-05
Epoch 5/10
271/271 [==============================] - 10s 37ms/step - loss: 0.3051 - accuracy: 0.8688 - val_loss: 0.3357 - val_accuracy: 0.8701 - lr: 5.0000e-05
Epoch 6/10
271/271 [==============================] - 10s 36ms/step - loss: 0.2706 - accuracy: 0.8878 - val_loss: 0.3282 - val_accuracy: 0.8644 - lr: 5.0000e-05
Epoch 7/10
271/271 [=========================

# Bakr CBOW Word2Vec Model

In [10]:
bakr_word2vec_model = load_word2vec_model("../word2vec_models//bakr/cbow/full_grams_cbow_300_twitter.mdl")

In [11]:
number_of_features = 300
max_len_str = 237
word2vec_path = "bakr/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(8643, 237, 300)
(8643, 71100)
[-1.135   -2.104    1.518    1.966   -1.1875  -0.8887   0.02058 -0.6045
  0.812    0.503   -0.561    1.524   -0.6035   0.00866 -0.825   -0.9634
 -0.643    0.3687   0.14     2.32    -0.665   -0.2158   0.3616   1.305
  0.144    0.522    0.5557  -2.898    0.1288   1.279   -0.3296   1.474
  0.725   -0.2395  -1.222   -0.2517   0.418   -1.958   -0.1783  -0.779
  0.08136 -1.038   -0.05753 -0.5967   0.1014   1.523   -0.4175  -0.562
 -0.9556  -1.049  ]
(177, 237, 300)
(177, 71100)
[-1.510e+00 -5.952e-01 -9.412e-02  7.877e-04  2.033e+00  1.078e+00
 -7.002e-01  6.519e-02  1.880e+00  1.279e-01  1.307e+00 -2.866e-01
  4.124e-01  8.623e-01  9.976e-01  1.078e+00  4.167e-01 -2.338e-01
  5.049e-01  4.478e-01 -3.826e-01  2.027e+00 -1.108e+00 -4.668e-01
  2.443e+00  1.760e+00  6.851e-01 -1.512e+00  1.218e+00  2.974e-01
 -2.649e-01 -8.027e-01  5.322e-01 -6.064e-01  3.132e-01 -1.330e+00
  5.669e-01 -1.406e+00 -4.509e-01  7.573e-01  4.875e-01 -2.795e+00
 -1.024e+00 -6.104e-01 

# With  Rmsprob and  Batch Normalization

In [12]:
callbacks_ = keras_callbacks(word2vec_type="bakr_cbow_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

./../ml_models_saved/dl_models/tensor_logs/run_2022_05_09_23_42_14_bakr_cbow_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_2 (Batc  (None, 237, 300)         1200      
 hNormalization)                                                 
                                                                 
 lstm_1 (LSTM)               (None, 237, 25)           32600     
                                                                 
 batch_normalization_3 (Batc  (None, 237, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout_1 (Dropout)         (None, 237, 25)           0         
                                                                 
 flatten_1 (Flatten)         (None, 5925)              

In [13]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10
271/271 [==============================] - 14s 39ms/step - loss: 0.9720 - accuracy: 0.6112 - val_loss: 0.6660 - val_accuracy: 0.7401 - lr: 5.0000e-05
Epoch 2/10
271/271 [==============================] - 10s 38ms/step - loss: 0.5625 - accuracy: 0.7455 - val_loss: 0.4405 - val_accuracy: 0.7910 - lr: 5.0000e-05
Epoch 3/10
271/271 [==============================] - 10s 37ms/step - loss: 0.4529 - accuracy: 0.8033 - val_loss: 0.3958 - val_accuracy: 0.8249 - lr: 5.0000e-05
Epoch 4/10
271/271 [==============================] - 10s 37ms/step - loss: 0.3816 - accuracy: 0.8392 - val_loss: 0.3729 - val_accuracy: 0.8475 - lr: 5.0000e-05
Epoch 5/10
271/271 [==============================] - 10s 36ms/step - loss: 0.3384 - accuracy: 0.8624 - val_loss: 0.3355 - val_accuracy: 0.8418 - lr: 5.0000e-05
Epoch 6/10
271/271 [==============================] - 10s 37ms/step - loss: 0.3059 - accuracy: 0.8746 - val_loss: 0.3238 - val_accuracy: 0.8531 - lr: 5.0000e-05
Epoch 7/10
271/271 [==============

# Muhammed CBOW Word2Vec Model

In [14]:
muhammed_word2vec_model = load_word2vec_model("../word2vec_models/muhammed/cbow/w2v_CBOW_300_3_400_10.model")

In [15]:
number_of_features = 300
max_len_str = 237
word2vec_path = "muhammed/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(8643, 237, 300)
(8643, 71100)
[ 0.2268  -0.9756  -0.4565   0.9346   0.4282  -0.9824  -0.4766  -1.237
  1.305   -0.777   -0.2522   0.6216   0.4358   1.75    -0.996   -0.9746
 -0.118    1.038    0.2406   1.468   -0.6333  -0.355    1.129   -0.2583
 -0.1019  -0.02823 -0.2379   2.113    0.874    0.4878   0.07336  0.6147
  1.942    0.2979   1.115   -0.6704  -0.7065  -0.912    0.3728   0.4736
  0.748   -0.632    1.598    0.4553   0.656   -1.322    1.005   -0.376
  0.1926  -0.5146 ]
(177, 237, 300)
(177, 71100)
[ 0.832    0.2073   0.9546   2.123   -1.687    0.0622  -0.8013  -0.0675
 -0.3008   0.902    0.7485   0.264    0.947    1.629   -1.772   -0.0646
 -0.1963   1.31     0.325    0.6016   0.576   -0.6763  -0.612    0.01938
 -1.041   -1.348   -0.921    0.2272  -0.4575   0.636    1.286    0.3552
  2.49     0.2988   0.257    0.8784  -0.1746   0.4263  -0.5815   0.9062
 -0.2245   0.82    -1.285    0.3457   1.321    0.767   -1.027   -0.1918
 -1.423    0.8867 ]


# With  Rmsprob and  Batch Normalization

In [16]:
callbacks_ = keras_callbacks(word2vec_type="muhammed_cbow_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

./../ml_models_saved/dl_models/tensor_logs/run_2022_05_09_23_44_14_muhammed_cbow_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization_4 (Batc  (None, 237, 300)         1200      
 hNormalization)                                                 
                                                                 
 lstm_2 (LSTM)               (None, 237, 25)           32600     
                                                                 
 batch_normalization_5 (Batc  (None, 237, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 237, 25)           0         
                                                                 
 flatten_2 (Flatten)         (None, 5925)          

In [17]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10
271/271 [==============================] - 14s 40ms/step - loss: 1.0483 - accuracy: 0.5794 - val_loss: 0.9075 - val_accuracy: 0.7006 - lr: 5.0000e-05
Epoch 2/10
271/271 [==============================] - 9s 35ms/step - loss: 0.5939 - accuracy: 0.7184 - val_loss: 0.5432 - val_accuracy: 0.7345 - lr: 5.0000e-05
Epoch 3/10
271/271 [==============================] - 10s 35ms/step - loss: 0.4922 - accuracy: 0.7809 - val_loss: 0.5051 - val_accuracy: 0.7740 - lr: 5.0000e-05
Epoch 4/10
271/271 [==============================] - 9s 35ms/step - loss: 0.4254 - accuracy: 0.8087 - val_loss: 0.4714 - val_accuracy: 0.7910 - lr: 5.0000e-05
Epoch 5/10
271/271 [==============================] - 10s 36ms/step - loss: 0.3825 - accuracy: 0.8337 - val_loss: 0.4536 - val_accuracy: 0.8079 - lr: 5.0000e-05
Epoch 6/10
271/271 [==============================] - 10s 35ms/step - loss: 0.3515 - accuracy: 0.8504 - val_loss: 0.4388 - val_accuracy: 0.8136 - lr: 5.0000e-05
Epoch 7/10
271/271 [================

# Load best model & predict test set

In [18]:
strat_test_set = read_file("test/strat_test_set.csv")
strat_test_set.head()

,label,text
0,1,حساء خفافيش الصين ابداع القرف يعني
1,1,اجراءات هكذا استعدت مصر لمواجهه فيروس كورونا م...
2,1,ايش الفايده لابس كمامات ومو لابس قفازات
3,0,الجدير بالذكر ان في الصين بيستخدموا ال في كل ح...
4,0,ولو ترسل صواريخ الصين وقفنا دون برج الفيصليه


In [19]:
X_test_text = list(strat_test_set['text'])
y_test = strat_test_set['label'].values

In [20]:
X_test_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(X_test_text)

print("Before Tokenization : \n", X_test_text[:3])
print("="*50)
print("After Tokenization : \n", X_test_text_tokenized[:3])
print("="*50)

fullgram_X_test_text_tokenized = get_all_ngrams(X_test_text_tokenized)
print("full gram tokenization : \n", fullgram_X_test_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['حساء خفافيش الصين ابداع القرف يعني', 'اجراءات هكذا استعدت مصر لمواجهه فيروس كورونا مرايتي', 'ايش الفايده لابس كمامات ومو لابس قفازات']
After Tokenization : 
 [['حساء', 'خفافيش', 'الصين', 'ابداع', 'القرف', 'يعني'], ['اجراءات', 'هكذا', 'استعدت', 'مصر', 'لمواجهه', 'فيروس', 'كورونا', 'مرايتي'], ['ايش', 'الفايده', 'لابس', 'كمامات', 'ومو', 'لابس', 'قفازات']]
full gram tokenization : 
 [['حساء', 'خفافيش', 'الصين', 'ابداع', 'القرف', 'يعني', 'حساء_خفافيش', 'خفافيش_الصين', 'الصين_ابداع', 'ابداع_القرف', 'القرف_يعني', 'حساء_خفافيش_الصين', 'خفافيش_الصين_ابداع', 'الصين_ابداع_القرف', 'ابداع_القرف_يعني'], ['اجراءات', 'هكذا', 'استعدت', 'مصر', 'لمواجهه', 'فيروس', 'كورونا', 'مرايتي', 'اجراءات_هكذا', 'هكذا_استعدت', 'استعدت_مصر', 'مصر_لمواجهه', 'لمواجهه_فيروس', 'فيروس_كورونا', 'كورونا_مرايتي', 'اجراءات_هكذا_استعدت', 'هكذا_استعدت_مصر', 'استعدت_مصر_لمواجهه', 'مصر_لمواجهه_فيروس', 'لمواجهه_فيروس_كورونا', 'فيروس_كورونا_مرايتي'], ['ايش', 'الفايده', 'لابس', 'كمامات', 'ومو', 'لابس', 'قفاز

In [22]:
rezk_model = keras_load_model("../ml_models_saved/dl_models/run_with_rezk_cbow_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )
bakr_model = keras_load_model("../ml_models_saved/dl_models/run_with_bakr_cbow_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )
muhammed_model = keras_load_model("../ml_models_saved/dl_models/run_with_muhammed_cbow_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )

X_test_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)

keras_f1_score_result(rezk_model, X_test_embed_matrix, y_test)
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(bakr_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)
keras_f1_score_result(bakr_model, X_test_embed_matrix, y_test)
print("="*50)

X_test_embed_matrix = text_to_matrix_using_word2vec(muhammed_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)
keras_f1_score_result(muhammed_model, X_test_embed_matrix, y_test)

(180, 237, 300)
(180, 71100)
[ 0.2527   0.522    0.1779   0.4744  -0.2773   0.06058 -0.129   -0.2115
  0.2888   0.6753   0.1991   0.4854   0.887    0.1575   0.04596  0.02391
  0.393    0.4778  -0.2332  -0.2058  -0.5474  -0.7773  -0.11176 -0.1393
 -0.3208  -0.1919  -0.277   -1.303    0.1914  -0.503    0.1576  -0.4834
 -0.1271  -0.799    1.056    0.5728  -0.7295  -0.4622  -0.3062  -0.05975
 -0.651    0.2406   0.12335  0.547    0.3616   0.00967  0.6387   0.6045
  0.07007 -0.457  ]
===================== Validate Result =====================
F1 score is:  0.9277777777777778
(180, 237, 300)
(180, 71100)
[ 0.1781   0.0907  -0.855    0.3633  -0.117   -0.253   -0.3142   0.0964
  0.9585  -0.1278  -0.10754  0.962    1.264   -1.41    -0.01978 -0.5273
 -0.8696  -0.364    1.239   -1.581   -0.75    -0.6987  -0.3423   0.4714
  0.2732  -0.2334   0.575   -1.28    -0.2057   0.1677  -1.732    0.969
  0.188   -0.746   -0.961   -0.794    0.2544  -0.10864  0.3572  -0.454
 -0.9976  -0.3215  -0.371   -0.6963  

0.9